In [83]:
import tensorflow as tf
import importlib
import os
import pickle
from models import *
from dataprocessor import *
from tqdm import tqdm
from change_point_detection import *
def reload_custom_libs():
    import loss_functions
    import models
    import dataprocessor
    importlib.reload(loss_functions)
    importlib.reload(models)
    importlib.reload(dataprocessor)

In [38]:
macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126

folder_path = "data"
files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]
files = [
    file
    for file in files
    if file
    not in (
        "CC00.NYB.xlsx",
        "LB00.CME.xlsx",
        "ES00.CME.xlsx",
        "NQ00.CME.xlsx",
        "YM00.CBT.xlsx",
        "SP00.CME.xlsx",
    )
]
data_list = []


data_list = process_data_list(files, macd_timescales, rtn_timescales, test=True)

处理文件中。。: 100%|██████████| 11/11 [00:51<00:00,  4.71s/it]


In [39]:
gaussion_process_list = []
for data in tqdm(data_list):
    price_series = data["close"]
    target = price_series.to_numpy().reshape((-1, 1))
    segment_list = get_segment_points(target, l_max=63, mu=0.999)
    segment_list = [data.iloc[start : end, :] for start, end in segment_list]
    gaussion_process_list.extend(segment_list)

100%|██████████| 11/11 [29:19<00:00, 159.99s/it]


In [59]:
target_set, labels, map = generate_tensors(data_list, timesteps, encoder_type = "one-hot", return_map=True)
target_set, labels, map
feature_sequences_tensor, dates_tensor, side_info_tensor = target_set
rtn_next_day, std = labels

生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:00<00:00, 24.35it/s]

one-hot 编码中...


In [98]:
date_list[0].dt.strftime("%Y-%m-%d")

AttributeError: 'numpy.datetime64' object has no attribute 'dt'

In [99]:
feature_cols = gaussion_process_list[0].columns.to_list()
for col in ("date", "sigma", "side_info", "close", "rtn"):
    feature_cols.remove(col)
    
features = [df[feature_cols] for df in gaussion_process_list if len(df) > 0]
date_list = [str(df["date"].values[-1]) for df in gaussion_process_list if len(df) > 0]
side_info = [df["side_info"] for df in gaussion_process_list if len(df) > 0]
rtn_next_day = [df["rtn_next_day"] for df in gaussion_process_list if len(df) > 0]
std = [df["sigma"] for df in gaussion_process_list if len(df) > 0]

side_info_tensor = tf.ragged.constant(side_info, dtype=tf.string)
dates_tensor = tf.constant(date_list, dtype=tf.string)
feature_sequences_tensor = tf.ragged.constant(features, dtype=tf.float32)

TypeError: Cannot convert ['rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', 'rtn_next_day', 'macd_8_24', 'macd_16_28', 'macd_32_96', 'rtn_1', 'rtn_21', 'rtn_63', 'rtn_126', 'rtn_252', '